In [ ]:
!git clone https://github.com/luisgarrote/Minimouse.git
%cd Minimouse
!pip install -e .

In [ ]:
import minimouse
from google.colab import output
output.enable_custom_widget_manager()
disp=minimouse.Display(cells_w=5,cells_h=5)

In [ ]:
disp.show()

In [ ]:

from minimouse.controllers import *




def reset():
    """Utiilizar caso seja necessario limpar o estado de variaveis"""
    pass

def user_controller(obs):
    """
        Define o controlador do robot.
        Inputs: pose, laser, goal_cell.
    """
    x, y, th = obs["pose"]
    laser = obs["laser"]
    gx_cell, gy_cell = map(int, obs["goal_cell"])
    # Convert goal cell -> world coords (env provides this)
    gx, gy = disp.env._cell_center_world(gx_cell, gy_cell)
    pos = (float(x), float(y))
    goal = (float(gx), float(gy))
    # ----------------------------
    # Laser sectors (IMPORTANT: +angles = left, -angles = right)
    # ----------------------------
    n = len(laser)
    angles = np.linspace(-math.pi/2, math.pi/2, n)

    front_mask = np.abs(angles) < math.radians(10)
    left_mask  = angles >  math.radians(35)
    right_mask = angles < -math.radians(35)

    front = float(np.min(laser[front_mask])) if np.any(front_mask) else disp.env.max_range
    left  = float(np.min(laser[left_mask]))  if np.any(left_mask)  else disp.env.max_range
    right = float(np.min(laser[right_mask])) if np.any(right_mask) else disp.env.max_range
    # goal stop tolerance: env considers done at ~0.45*cell_size radius
    goal_stop = 0.45 * disp.env.cell_size



    d_goal = dist(pos, goal)
    if d_goal < goal_stop:   # chegou ao objectivo
        return 0.0, 0.0

    ################################################
    #      TODO
    #      Definir (v, w)
    #      Mapa disponivel em disp.env.occ
    ################################################
    # Convert (v, w) -> (vl, vr)
    # ----------------------------
    L = disp.env.wheel_base
    vl = v - 0.5 * L * w
    vr = v + 0.5 * L * w
    # Clip wheel speeds
    max_wheel = 6.0 * disp.env.cell_size
    vl = float(np.clip(vl, -max_wheel, +max_wheel))
    vr = float(np.clip(vr, -max_wheel, +max_wheel))
    return vl, vr

disp.callbacks["user_controller"]=user_controller
disp.callbacks["reset_hook"]=reset
